In [13]:
import pandas as pd
import numpy as np
import pingouin as pg
import requests
import datetime

In [2]:
def read_csv(url, sep=','):
    key = url
    api_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key="+key
    response = requests.get(api_url)
    download_url = response.json()['href']
    df = pd.read_csv(download_url, sep=sep)
    return df

In [4]:
problem1_reg = read_csv('https://disk.yandex.ru/d/Xn78-_wgoIT6dA', sep = ';')
problem1_auth = read_csv('https://disk.yandex.ru/d/BxnJZSKyNVQt_A', sep = ';')

In [57]:
total_users = problem1_auth.merge(problem1_reg, on = 'uid')

In [58]:
total_users.auth_ts = total_users.auth_ts.apply(lambda x: datetime.datetime.utcfromtimestamp(x))
total_users.reg_ts = total_users.reg_ts.apply(lambda x: datetime.datetime.utcfromtimestamp(x))

C:\Users\kirill\AppData\Local\Temp\ipykernel_20584\3018260412.py:1: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  total_users.auth_ts = total_users.auth_ts.apply(lambda x: datetime.datetime.utcfromtimestamp(x))
C:\Users\kirill\AppData\Local\Temp\ipykernel_20584\3018260412.py:2: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  total_users.reg_ts = total_users.reg_ts.apply(lambda x: datetime.datetime.utcfromtimestamp(x))


In [59]:
total_users['day'] = pd.to_datetime(total_users.reg_ts.dt.date)
total_users['day_auth'] = pd.to_datetime(total_users.auth_ts.dt.date)

In [71]:
total_users['timedelta'] = (total_users.day_auth - total_users.day).dt.days

In [168]:
total_users.timedelta.unique()

array([   0,    3,    9, ..., 5212, 5253, 5353])

In [163]:
cohort = total_users.query('timedelta > 0').groupby(['day', 'timedelta'], as_index=False).uid.nunique()

In [169]:
cohort.head()

,day,timedelta,uid
0,1999-07-22,3,1
1,1999-07-22,9,1
2,1999-07-22,14,1
3,1999-07-22,20,1
4,1999-07-22,23,1


In [166]:
cohort_pivot = cohort.pivot(index = 'day', columns = 'timedelta', values = 'uid')

In [167]:
cohort_pivot

timedelta,1,2,3,4,5,6,7,8,9,10,...,7701,7704,7706,7709,7712,7716,7720,7721,7727,7729
day,,,,,,,,,,,,,,,,,,,,,
1999-07-22,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2002-05-05,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-07-16,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-10-18,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-11-15,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-18,31.0,74.0,69.0,85.0,61.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-19,30.0,58.0,77.0,57.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-20,40.0,71.0,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
cohort_matrix = cohort.pivot(columns = 'timedelta', index = 'day', values = 'uid')

In [149]:
cohort_size = cohort_matrix[1]

In [155]:
retention = cohort_matrix.divide(cohort_size, axis=0)

In [158]:
cohort_matrix

timedelta,1,2,3,4,5,6,7,8,9,10,...,7701,7704,7706,7709,7712,7716,7720,7721,7727,7729
day,,,,,,,,,,,,,,,,,,,,,
1999-07-22,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2002-05-05,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-07-16,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-10-18,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-11-15,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-18,31.0,74.0,69.0,85.0,61.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-19,30.0,58.0,77.0,57.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-20,40.0,71.0,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
